In [24]:
import wandb

wandb.init(project="Dataset-Analysis", entity="anton-peter")
wandb.run.name = f"Round performance"



embryo_time_execeution = [2.867, 3.633, 3.067, 3.033 ,3.067, 3.217, 2.917, 4.433,3.083,  2.5,   2.85,   2.633, 3.05, 2.667, 2.917, 2.917, 2.533, 3.483, 3.067, 2.517, 3.033, 2.633, 2.65]   
client_keys = [id+1 for id in range(23)]

performance_dict = dict(zip(client_keys, embryo_time_execeution))
data = [[item[0], int(item[1]*60)] for i, item in enumerate(performance_dict.items())]

table = wandb.Table(data=data, columns=["client-id", "Round-time"])
wandb.log({"Client round performance Embryo" : wandb.plot.bar(table, "client-id",
           "Round-time", title="Client round performance Embryo")})


mnist_time_execeution = [0.45,0.433,0.45,0.433,0.433,0.4167,0.4333,0.45,0.45,0.4833,0.4833,0.4833,0.4833,0.4833,0.4833,0.4833,0.4833,0.4833,0.4833,0.4833,0.4833,0.4833,0.4667]
client_keys = [id+1 for id in range(23)]

performance_dict2 = dict(zip(client_keys, mnist_time_execeution))
data2 = [[item[0], int(item[1]*60)] for i, item in enumerate(performance_dict2.items())]

table = wandb.Table(data=data2, columns=["client-id", "Round-time"])
wandb.log({"Client round performance MNIST" : wandb.plot.bar(table, "client-id",
           "Round-time", title="Client round performance MNIST")})

wandb.finish()

2022-05-28 21:37:52.603527: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-28 21:37:52.727527: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


### EVAL ON clients with 100 + samples

In [12]:
from tensorflow import keras
from keras.datasets import mnist
import numpy as np
import sys

sys.path.append('../..')
import utils.models.vitrolife_model as Models
import utils.dataloaders.vitrolife_dataloader as DataloaderBla


#load model
#model_name= "../../experiments/vitrolifeFederated/models/Embryo-Federated-B16-R300-E1-HIGH-LR-OverSampling-DA/R-89"
model_name= "../../experiments/vitrolifeCentralized/models/Embryo-Centralized-B16-E100/Epoch-99"
model_from_disk = keras.models.load_model(f"{model_name}")

all_results = []
list_train = []

batch_size = 16
number_of_clients=23
for client_id in range(23):
    path =  "/mnt/data/vitroLifeDataset"
    test_loader = DataloaderBla.VitroLifeDataloader(path, batch_size, dataset_split="Test", clinic_ID=client_id)
    result = model_from_disk.evaluate(test_loader)
    
    training_loader = DataloaderBla.VitroLifeDataloader("/mnt/data/vitroLifeDataset", batch_size , dataset_split="Train", data_augmentation = "False", balancing_strategy="None", clinic_ID=client_id)
    training_data = training_loader.__len__()
    list_train.append(training_data*batch_size)

    
    if list_train[client_id] >= 150:
        all_results.append(result)
    else:
        print("skipping low")
        number_of_clients = number_of_clients -1


# acum all 23 clients metrics
aggregated_results = [0,0,0,0,0,0,0,0,0,0]    
for partial_results in all_results:
    for index, metric in enumerate(partial_results):
            aggregated_results[index] += metric
        
        
# take mean        
for index, agg_res in enumerate(aggregated_results):
    aggregated_results[index] = aggregated_results[index]/number_of_clients
    
     
    
standart_deviation = []    
# collect std    
for i in range(10):
    metrics_groups = [item[i] for item in all_results]
    standart_deviation.append(np.std(metrics_groups))

print("std")
print(standart_deviation)
    

metric_names =  ['loss', 'tp', 'fp', 'tn', 'fn', 'accuracy', 'presision', 'recall', 'auc', 'prc']
for i, final_res in enumerate(aggregated_results):
    print(f"{metric_names[i]} {final_res} +/- {standart_deviation[i]}")
    

#print(f"Experiment model: {model_name}")
#print(f"loss={result[0]}\n tp={result[1]}\n fp={result[2]}\n tn={result[3]}\n fn={result[4]}\n accuracy={result[5]}\n presision={result[6]}\n recall={result[7]}\n auc={result[8]}\n prc={result[9]}")

1/1 [==============================] - 0s 162ms/step - loss: 0.6750 - tp: 1.0000 - fp: 0.0000e+00 - tn: 6.0000 - fn: 4.0000 - accuracy: 0.6364 - precision: 1.0000 - recall: 0.2000 - auc: 0.6167 - prc: 0.6928
skipping low
1/1 [==============================] - 0s 192ms/step - loss: 0.6898 - tp: 4.0000 - fp: 7.0000 - tn: 4.0000 - fn: 1.0000 - accuracy: 0.5000 - precision: 0.3636 - recall: 0.8000 - auc: 0.6455 - prc: 0.5187
skipping low
1/1 [==============================] - 0s 168ms/step - loss: 0.6758 - tp: 3.0000 - fp: 5.0000 - tn: 6.0000 - fn: 1.0000 - accuracy: 0.6000 - precision: 0.3750 - recall: 0.7500 - auc: 0.5000 - prc: 0.2441
skipping low
1/1 [==============================] - 0s 148ms/step - loss: 0.5847 - tp: 2.0000 - fp: 2.0000 - tn: 6.0000 - fn: 1.0000 - accuracy: 0.7273 - precision: 0.5000 - recall: 0.6667 - auc: 0.8333 - prc: 0.6594
skipping low
1/1 [==============================] - 0s 149ms/step - loss: 0.5893 - tp: 0.0000e+00 - fp: 1.0000 - tn: 5.0000 - fn: 2.0000 - ac